In [1]:
import torch
from torch import nn
from torch import functional as F
import matplotlib.pyplot as plt
import numpy as np
from livelossplot import PlotLosses
from Tools.NNtools import *
import seaborn as sns, numpy as np
import functools
from scipy.spatial.distance import squareform
import math
from scipy.stats import invwishart

### IMPORTS

In [2]:
# Find what device to work with
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
device 

device(type='cuda', index=2)

In [3]:
torch.set_default_dtype(torch.float64)

# Model principal

In [4]:
D = 1000
g = 3

In [5]:
I = np.eye(D)

In [6]:
iw = invwishart(df=D+g, scale=I*g)

In [7]:
mu = torch.zeros(D, device=device)
sigma = torch.tensor(iw.rvs(), device=device)

In [8]:
sigma;

In [9]:
Target = torch.distributions.MultivariateNormal(mu, sigma)

In [10]:
Target.entropy()

tensor(-992.8223, device='cuda:2')

In [ ]:
def logposterior(y):
    return Target.log_prob(y) 

In [ ]:
N = 1000

In [ ]:
x = Target.sample((N,))

In [ ]:
x.shape

In [ ]:
E = torch.tensor(0.0)
for r in range(N):
    E += logposterior(x[r,:].unsqueeze(0)).squeeze()
E /= N
-E

In [ ]:
N = 10000

In [ ]:
x = Target.sample((N,))

In [ ]:
[KNNE(Target.sample((N,)),1), KNNE(Target.sample((N,)),1), KNNE(Target.sample((N,)),1)]

In [ ]:
def KNNE(samples,k=1):  
    nb_samples = samples.shape[0]
    d = samples.shape[1]
    D = torch.cdist(samples,samples)
    a = torch.topk(D, k=k+1, dim=0, largest=False, sorted=True)[0][k].clamp(torch.finfo().eps,float('inf'))
    lcd = (d/2.0*torch.log(torch.tensor(math.pi))) - torch.lgamma(torch.tensor(1.0 + d/2.0))
    return torch.digamma(torch.tensor(nb_samples).float()).double() - torch.digamma(torch.tensor(k).float()).double() + lcd + d/nb_samples*torch.sum(torch.log(a))

In [ ]:
k = 1
Dist = torch.cdist(x,x,p=2.0)
a = torch.topk(Dist, k=k+1, dim=0, largest=False, sorted=True)[0][1]

In [ ]:
lcd = (D/2.0*torch.log(torch.tensor(np.pi))) - torch.lgamma(torch.tensor(1.0 + D/2.0))
lcd

In [ ]:
D/N*torch.sum(torch.log(a))

In [ ]:
0.57721

In [ ]:
torch.log(torch.tensor(N).float()).double()

In [ ]:
D/N*torch.sum(torch.log(a)) + lcd + 0.57721 + torch.log(torch.tensor(N).float()).double()

In [ ]:
def Estimate():
    N = 1000000
    xstar = Target.sample((1,))
    X = Target.sample((N,))
    Dist = torch.cdist(xstar,X)
    return D*torch.log(Dist.min()) + lcd + 0.57721 + torch.log(torch.tensor(N).float()).double()

In [ ]:
L = [Estimate() for _ in range(10)]
torch.stack(L).mean()

In [ ]:
M = 1000000
m = torch.tensor(np.inf, device=device)

In [ ]:
xstar = Target.sample((1,))

In [ ]:
for _ in range(1000):
    m =  torch.min(torch.cdist(xstar,Target.sample((M,))).min(), m)

In [ ]:
m

In [33]:
neighborhood = 2*D

In [34]:
N = 10000

In [35]:
x = Target.sample((N,))
Dist = torch.cdist(x,x)

In [36]:
(TopDist, TopIdx) = torch.topk(Dist, k=1+neighborhood, dim=0, largest=False, sorted=True)

In [37]:
compactness = torch.mean(TopDist,dim=0)

In [38]:
i = torch.argmin(compactness)

In [39]:
idx = TopIdx[:,i]

In [40]:
TopIdx

tensor([[   0,    1,    2,  ..., 9997, 9998, 9999],
        [ 221, 5628, 3056,  ..., 4652, 7783, 2052],
        [2340, 9140, 2201,  ..., 2699, 3514, 8629],
        ...,
        [1692, 5850, 6194,  ..., 1887, 5711, 3898],
        [8254, 3264, 1588,  ..., 9652,  434, 7023],
        [7848, 4165,    8,  ..., 2106, 9267, 9439]], device='cuda:2')

In [41]:
F = x[idx,:]

In [42]:
m = torch.mean(x[idx,:], dim=0).repeat(F.shape[0], 1)

In [43]:
F.shape

torch.Size([2001, 1000])

In [44]:
G = (F - m)

In [45]:
C = (G.unsqueeze(2)*G.unsqueeze(1)).sum(dim=0)/(F.shape[0]-1)

RuntimeError: CUDA out of memory. Tried to allocate 14.91 GiB (GPU 2; 10.92 GiB total capacity; 2.21 GiB already allocated; 1.70 GiB free; 6.40 GiB cached)

In [31]:
localGaussian = torch.distributions.MultivariateNormal(torch.zeros(D, device=device), C)

In [32]:
localGaussian.entropy()

tensor(-1498.0080, device='cuda:2')